In [1]:
import tensorflow as tf
import os
import tarfile
import requests

In [2]:
#模型下载地址
inception_pretrain_model_url='http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'

#模型存放地址
#创建文件夹的名字及路径,os.path.exists()函数用来检验给出的路径是否真地存在 返回bool
inception_pretrain_model_dir="inception_model"
if not os.path.exists(inception_pretrain_model_dir):
    #makedir(path):创建文件夹，注：创建已存在的文件夹将异常
    os.makedirs(inception_pretrain_model_dir)  

#获取文件名以及文件路径
#inception_pretrain_model_url='http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
#filename取以/分开的最后一个字符串即inception-2015-12-05.tgz
filename=inception_pretrain_model_url.split('/')[-1]
#将两个路径连接起来：inception_model\inception-2015-12-05.tgz
filepath=os.path.join(inception_pretrain_model_dir, filename)

#下载模型
# 如果路径名不存在（这里指的是路径下的内容）的话，就开始下载文件
if not os.path.exists(filepath):
    print("download:", filename)
    #requests.get从指定http网站上下载内容
    r=requests.get(inception_pretrain_model_url, stream=True)
    # 用with语句来打开文件，就包含关闭的功能。wb是写二进制文件，由于文件过大，批量写（这里是压缩包）
    with open(filepath, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
print("finish: ",filename)

#tarfile解压文件 
#其中，解压出来的文件其中的classify_image_graph_def.pb 文件就是训练好的Inception-v3模型
#imagenet_synset_to_human_label_map.txt是类别文件
tarfile.open(filepath, 'r:gz').extractall(inception_pretrain_model_dir)

#模型结构存放文件，创建inception_log文件夹
log_dir='inception_log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

#classify_image_graph_def.pb为google训练好的模型
inception_graph_def_file=os.path.join(inception_pretrain_model_dir, 'classify_image_graph_def.pb')

with tf.Session() as sess:
    #创建一个图来保存google训练好的模型，图片所在路径inception_graph_def_file 二进制方式读取文件
    #inception_pretrain_model_dir=inception_model\ 里面存放多个文件
    with tf.gfile.FastGFile(inception_graph_def_file, 'rb') as f:
        # 新建GraphDef文件，用于临时载入模型中的图
        graph_def=tf.GraphDef()
        # GraphDef加载模型中的图   
        graph_def.ParseFromString(f.read())
        # 在空白图中加载GraphDef中的图
        tf.import_graph_def(graph_def, name='')
    #指定log_dir=inception_log文件保存图的结构
    writer=tf.summary.FileWriter(log_dir, sess.graph)
    writer.close()

download: inception-2015-12-05.tgz
finish:  inception-2015-12-05.tgz
Instructions for updating:
Use tf.gfile.GFile.
